## **ETL**

In [1]:
import requests
import pandas as pd
!pip install sqlalchemy pymysql
!pip install sqlalchemy pyodbc
import time
import pyodbc
from datetime import datetime
from sqlalchemy import create_engine, NVARCHAR, FLOAT, inspect
import logging
from tqdm import tqdm

## **Connection to the SQL db (SINK)**


In [13]:
# SQL Server details

SERVER = '(localdb)\\localdb'
DATABASE = 'Irceline'
DRIVER = 'ODBC Driver 18 for SQL Server'
CONNECTION_STRING = f'mssql+pyodbc://{SERVER}/{DATABASE}?driver={DRIVER.replace(" ", "+")}&trusted_connection=yes'
engine = create_engine(CONNECTION_STRING, fast_executemany=True)

## **API**

In [3]:
url = "https://geo.irceline.be/sos/api/v1/stations"
response = requests.get(url)
stations = response.json()

In [4]:
url1 = "https://geo.irceline.be/sos/api/v1/timeseries"
response1 = requests.get(url1)
timeseries = response1.json()

In [5]:
url2 = "https://geo.irceline.be/sos/api/v1/categories"
response2 = requests.get(url2)
category = response2.json()

In [18]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# API Configuration
TIMESERIES_URL = "https://geo.irceline.be/sos/api/v1/timeseries"


def fetch_api_data(url):
    logger.info(f"Fetching URL: {url}")
    try:
        response = requests.get(url, headers={"Accept": "application/json"}, timeout=30)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        logger.warning(f"API request failed for {url}: {str(e)}")
        return None
    
def load_measurements():
    ts_df = pd.read_sql("SELECT id AS timeseries_id FROM timesseriesdata", engine)

    for _, row in tqdm(ts_df.iterrows(), total=ts_df.shape[0], desc="Fetching measurements"):
        ts_id = row['timeseries_id']
        url = f"{TIMESERIES_URL}/{ts_id}/getData?timespan=2025-01-01TZ/2025-07-06TZ"
        logger.info(f"Fetching timeseries data from: {url}")

        values = fetch_api_data(url)
        if not values or 'values' not in values:
            logger.warning(f"No 'values' found for timeseries {ts_id}")
            continue

        records = []
        for point in values['values']:
            timestamp_raw = point.get('timestamp')
            value_raw = point.get('value')

            if value_raw is None or timestamp_raw is None:
                continue

            try:
                timestamp = datetime.utcfromtimestamp(timestamp_raw / 1000)
                value = float(value_raw)
                records.append({
                    "timeseries_id": ts_id,
                    "timestamp": timestamp,
                    "value": value
                })
            except Exception as e:
                logger.warning(f"Skipping invalid data point in timeseries {ts_id}: {e}")
                continue

        if records:
            pd.DataFrame(records).to_sql("measurements", engine, if_exists="append", index=False)

# === ETL Run ===
load_measurements()
print("✅ Measurements data loaded.")


Fetching measurements:   0%|          | 0/658 [00:00<?, ?it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/6522/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/6522/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:   0%|          | 1/658 [00:00<04:30,  2.43it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/10892/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/10892/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:   0%|          | 2/658 [00:01<07:27,  1.47it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/10893/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/10893/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching me

INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/11000/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:   4%|▍         | 25/658 [00:19<08:41,  1.21it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/11001/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/11001/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:   4%|▍         | 26/658 [00:20<08:55,  1.18it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/11002/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/11002/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:   4%|▍         | 27/658 [00:21<09:24,  1.12it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/11003/getData?timespan=2025-01-01TZ/2025-07-06T

Fetching measurements:   7%|▋         | 49/658 [00:33<07:14,  1.40it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/10706/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/10706/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:   8%|▊         | 50/658 [00:34<07:32,  1.34it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/10742/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/10742/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:   8%|▊         | 51/658 [00:35<07:38,  1.32it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/10747/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/10747/getData?timespan=2025-01-01TZ/2025-07-06T

INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/11053/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:  11%|█         | 74/658 [00:54<06:33,  1.49it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/11058/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/11058/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:  11%|█▏        | 75/658 [00:54<06:43,  1.44it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/11066/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/11066/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:  12%|█▏        | 76/658 [00:55<06:51,  1.41it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/6500/getData?timespan=2025-01-01TZ/2025-07-06TZ

INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/6521/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:  15%|█▌        | 99/658 [01:09<05:02,  1.85it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/6518/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/6518/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:  15%|█▌        | 100/658 [01:09<04:24,  2.11it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/100019/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/100019/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:  15%|█▌        | 101/658 [01:10<05:03,  1.84it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/100026/getData?timespan=2025-01-01TZ/2025-07-0

...

INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/6559/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:  86%|████████▌ | 567/658 [07:28<00:47,  1.92it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/6572/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/6572/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:  86%|████████▋ | 568/658 [07:29<00:58,  1.54it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/6613/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/6613/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:  86%|████████▋ | 569/658 [07:30<00:48,  1.84it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/6620/getData?timespan=2025-01-01TZ/2025-07-06TZ
I

INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/6996/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:  90%|████████▉ | 592/658 [07:44<00:49,  1.33it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/7004/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/7004/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:  90%|█████████ | 593/658 [07:45<00:51,  1.27it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/7014/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/7014/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:  90%|█████████ | 594/658 [07:46<00:54,  1.18it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/7045/getData?timespan=2025-01-01TZ/2025-07-06TZ
I

Fetching measurements:  94%|█████████▎| 616/658 [07:57<00:27,  1.54it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/10967/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/10967/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:  94%|█████████▍| 617/658 [07:57<00:22,  1.84it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/10968/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/10968/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:  94%|█████████▍| 618/658 [07:58<00:26,  1.52it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/10969/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/10969/getData?timespan=2025-01-01TZ/2025-07-

INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/99929/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:  97%|█████████▋| 641/658 [08:17<00:16,  1.05it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/99932/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/99932/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:  98%|█████████▊| 642/658 [08:18<00:15,  1.02it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/99935/getData?timespan=2025-01-01TZ/2025-07-06TZ
INFO:__main__:Fetching URL: https://geo.irceline.be/sos/api/v1/timeseries/99935/getData?timespan=2025-01-01TZ/2025-07-06TZ
Fetching measurements:  98%|█████████▊| 643/658 [08:19<00:14,  1.00it/s]INFO:__main__:Fetching timeseries data from: https://geo.irceline.be/sos/api/v1/timeseries/99938/getData?timespan=2025-01-01TZ/2025-07-

✅ Measurements data loaded.


## **Data Transformation**

In [163]:
flattened = []
for feature in stations:
    flattened.append({
        "id": feature["properties"]["id"],
        "label": feature["properties"]["label"],
        "longitude": feature["geometry"]["coordinates"][0],
        "latitude": feature["geometry"]["coordinates"][1],
        "altitude": feature["geometry"]["coordinates"][2] if len(feature["geometry"]["coordinates"]) > 2 else None,
        "geometry_type": feature["geometry"]["type"],
        "feature_type": feature["type"]
    })

# Convert to DataFrame
df_stations = pd.DataFrame(flattened)

# Extract just the city name (second part after first hyphen)
df_stations['city'] = df_stations['label'].str.split(' - ').str[1]

# Optionally, you can clean up the city name by removing any trailing whitespace
df_stations['city'] = df_stations['city'].str.strip()

# If you want to keep only the city column and drop the original label:
# df_stations.drop('label', axis=1, inplace=True)

# Or if you want to keep both:
df_stations = df_stations[['id', 'city', 'latitude', 'longitude']]
df_stations[['city_main', 'city_district']] = (
    df_stations['city']
    .str.extract(r'^([^(]+)(?:\s*\((.*)\))?$')
    .fillna('')  # Replace NaN (no match) with empty string
)
df_stations.drop(['city'], axis=1, inplace = True)
mask = df_stations['city_main'] == 'Arts-Loi'
df_stations.loc[mask, 'city_district'] = 'Arts-Loi ' + df_stations.loc[mask, 'city_district']
df_stations.loc[mask, 'city_main'] = 'Bruxelles'
df_stations['city_main'] = df_stations['city_main'].str.strip()
df_stations['city_district'] = df_stations['city_district'].str.strip()
df_stations['city_main'] = df_stations['city_main'].replace('Brussel', 'Bruxelles')

df_stations.to_sql(
    name='stations',        # target table name
    con=engine,             # SQLAlchemy engine
    if_exists='replace',    # 'replace', 'append', or 'fail'
    index=False,            # do not write DataFrame index as a column
)

-1

In [136]:
# Flatten timeseries

flat_data_list = []

for data in timeseries:
    flat_data = {
        "id": data["id"],
        "label": data["label"],
        "uom": data["uom"],
        "station_id": data["station"]["properties"]["id"],
        "station_label": data["station"]["properties"]["label"],
        "longitude": data["station"]["geometry"]["coordinates"][0],
        "latitude": data["station"]["geometry"]["coordinates"][1],
        "altitude": data["station"]["geometry"]["coordinates"][2],
        "geometry_type": data["station"]["geometry"]["type"],
        "station_type": data["station"]["type"]
    }
    flat_data_list.append(flat_data)

df_ts = pd.DataFrame(flat_data_list)

# Split the label column into multiple components
df_ts[['parameter', 'id_part', 'device_type', 'station_info']] = df_ts['label'].str.split(' - ', n=3, expand=True)
print(df_ts.head(5))

# Further split the measurement into name and ID (since ID is already a column)
df_ts['parameter_name'] = df_ts['parameter'].str.replace(r' \d+$', '', regex=True)
df_ts['parameter_id'] = df_ts['parameter'].str.extract(r'(\d+)$')
df_stations.drop(['station_label', 'longitude', 'latitude', 'altitude', 'geometry_type', 'station_info'], axis=1, inplace = True)

print(df_ts.head(5))
df_ts.to_sql(
    name='timesseriesdata',        # target table name
    con=engine,             # SQLAlchemy engine
    if_exists='replace',    # 'replace', 'append', or 'fail'
    index=False,            # do not write DataFrame index as a column
)

      id                                              label    uom  \
0   6522  1,2-XYLENE O-XYLENE 6522 - btx, o-xyleen - pro...  µg/m³   
1  10892  Ammonia 10892 - MINI DOAS - procedure, 42BO01 ...  µg/m³   
2  10893  Ammonia 10893 - MINI DOAS - procedure, 42OT02 ...  µg/m³   
3  10932  Ammonia 10932 - Unknown device - procedure, 43...  µg/m³   
4  10933  Ammonia 10933 - Unknown device - procedure, 43...  µg/m³   

   station_id                                      station_label  longitude  \
0        6522  41B006 - Bruxelles (Parlement UE) - 1,2-XYLENE...   4.374388   
1       10892             42BO01 - Bonheiden - Ammonia - feature   4.516484   
2       10893          42OT02 - Oud-Turnhout - Ammonia - feature   5.011793   
3       10932       43N063 - Corroy-Le-Grand - Ammonia - feature   4.668288   
4       10933               43N100 - Dourbes - Ammonia - feature   4.594763   

    latitude altitude geometry_type station_type                 parameter  \
0  50.838631      NaN     

-1

In [9]:
#CATEGORY

df_category = pd.DataFrame(category)

print(df_category)

df_category.to_sql(
    name='Category',        # target table name
    con=engine,             # SQLAlchemy engine
    if_exists='replace',    # 'replace', 'append', or 'fail'
    index=False,            # do not write DataFrame index as a column
)

       id                                    label
0     482                      1,2-XYLENE O-XYLENE
1      35                                  Ammonia
2   64102                    Atmospheric  Pressure
3      20                                  Benzene
4     391                             Black Carbon
5      71                           Carbon Dioxide
6      10                          Carbon Monoxide
7    4013                Elemental gaseous mercury
8     431                             Ethylbenzene
9     464                               M+P-xylene
10      8                         Nitrogen dioxide
11     38                        Nitrogen monoxide
12      7                                    Ozone
13  90517  Particle number concentration 10-800 nm
14   6002                Particulate Matter < 1 µm
15      5               Particulate Matter < 10 µm
16   6001              Particulate Matter < 2.5 µm
17  62106                        Relative Humidity
18      1                      

-1